# Monad

## Outline

* Incentive for Monad

* Definition of Monad type class

* Monads and do-notation
  - Revisiting do-notation in IO

  - Generalizing do-notation

* Monad examples

* Monad laws

In this lesson, we will learn about the Monad type classe and how you can use it. We will first show what kind of basic problem does the Monad type class solve. Then we will re-visit do-notation, show again how it works and generalize it for all type that have a Monad instance. We will show how you can create a Monad of your own and explain Monad laws. 

## Incentive for Monad

We covered until now two cases where the Functor and Applicative type class functions can help us. 

If your input data is in a `Maybe` cotext and you have the following functions:
```haskell
f1 :: a -> b
f2 :: Maybe (a -> b) 
```
then you can use the `<$>` and `<*>` operators to apply them to your data of type `Maybe a` and get data of type `Maybe b` returned. 

This works for the `Maybe` type because it has an instance of Functor and Applicative. 

You can also define your custom type and make it an instance of Functor and Applicative.

To describe it in a more general way: 
- we use Functors to solve the problem of mapping a function over a value inside a context

- we use Applicative to solve the problem of how to apply a function that is also inside the context.

But what if we have a value with a context `m a` and a funciton that takes a value without the context and returns it back again with the context: `f :: (a -> m b)`? 

Let's take an example for the Maybe type that provides the context for missing values:
```haskell
f3 :: a -> Maybe b
```

Below is an example with `Map` objects. We define book codes that map to book IDs. Then we define book prices that map to book codes.

We write a function that takes a book ID and returns a Maybe book prices. We use the `fromJust` and `isJust` functions from the **Data.Maybe** module.

In [ ]:
import Data.Maybe (fromJust, isJust)
import qualified Data.Map as Map

bookCodes :: Map.Map Int Int
bookCodes = Map.fromList $ zip [1,2,3] [2869, 7435, 1423]

bookPrices :: Map.Map Int String
bookPrices = Map.fromList $ zip [2869, 7435, 1423] ["15EUR","20USD","18JPY"]

getPriceFromID :: Int -> Maybe String
getPriceFromID n 
    | isJust bookID = Map.lookup (fromJust bookID) bookPrices
    | otherwise = Nothing
    where bookID = Map.lookup n bookCodes

print $ getPriceFromID 1

As stated in the previous lessons for functor and applicative, creating a manual solution for every type that represents a context is tedious.

The Monad type class provides a solution for solving the described problem above.

## Definition of the Monad type class

The definition of Monad type class is the following:
```haskell
class Applicative m => Monad m where
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  return :: a -> m a
  {-# MINIMAL (>>=) #-}
```

We see that Applicative is a superclass of Monad. The minimal complete definition requires just the bind operator `>>=`. 

The bind operator has exactly the type signature we were looking at in our previous example.

What it does is that it extracts the value from the context applies the function to it and returns the result.

The sequncing operator `>>` takes two values in a context and returns the second one. 

That makes sense when your values are IO actions that have side effects. Then you can chain those actions together. 

We notice that the `return` function does the same as the `pure` function from Applicative. It puts a value inside a context.

Because Applicative is a superclass of Monad you also need to define and instance for it when creating a monad instance. 

In the Monad instance you can then say `return = pure`. This is how the `return` function is defined in Haskell source code.

But you can also define `return` in the Monad type class and then when defining the Applicative type you say `pure = return`. 

The reason why the Monad type class contains `return` is because the Monad type class predates the Applicative type class.

### Simple example

The Maybe type has also an instance of Monad type class. Let's see how we can use the bind operator to rewrite the previous code example. 

We will use the `flip` function that reverses the order of input parameters for a function that takes in only two parameters.
```haskell
flip :: (a -> b -> c) -> b -> a -> c
```

In [ ]:
getPriceFromIDBind :: Int -> Maybe String
getPriceFromIDBind n = bookID >>= flip Map.lookup bookPrices
    where bookID = Map.lookup n bookIDs

print $ getPriceFromIDBind 1

The reason why to use the bind operator compared to our previous approach is that it's easier to read the code because it's generic. 

The same symbol serves the same purpose for all Monads. And some of Haskell existing type already have a Monad instance as Maybe, IO and lists.

So you get the monad operators which can work with those types for free.

In the next chapter we will show exaples how the sequencing operator and the return function work.

## Monads and do-notation

### Revisiting do-notation in IO

We have already explained how do-notation works on simple examples in lesson 11 that was about IO.

Let's first remember how the operators from the Monad type class work in the backgroup for do-notation in IO Monads.

Below a simple program that asks a user for his name and then prints back a greeting. Both functions do the same.

In [ ]:
hello :: IO () 
hello = 
  print "What is your name?" >> 
  getLine >>= 
  (\name -> return ("Hello " ++ name)) >>= 
  print

hello' :: IO () 
hello' = do 
  print "What is your name?" 
  name <- getLine 
  print ("Hello " ++ name) 

hello

As said in lesson 11 the `do` notation is just a syntactic sugar (a nicer way of writing the same thing) for the code that uses `>>` and `>>=` operators.

If you compare the code you see that the `<-` arrow is used to assign a variable that comes in a context to a variable without that context. 

In our case the `getLine` function returns an `IO String` and the variable `name` is just of type `String`. The statements are then chained together with the `>>` and `>>=` operators.

As you may remember you can also declare some variables inside a `do` block with a `let`. 

Below is the code that shows the `do` notation code and the desugared code that uses lambda functions and the Monad operators.

The functions `monadExample` and `monadExample'` do the same thing.

In [ ]:
monadExample :: IO Int 
monadExample = do 
  let a = return 1 :: IO Int 
      func var = return (var + 1) :: IO Int 
  a >>= func

monadExample' :: IO Int 
monadExample' = 
  (\a -> 
    a >>= 
      (\var -> 
        return (var + 1) :: IO Int 
      ) 
  ) (return 1 :: IO Int)

main :: IO ()
main = do
    var1 <- monadExample
    var2 <- monadExample'
    print var1
    print var2

main

The declaration of variables with the `let` operator is translated by using lambda functions in the background.

The `let` and `<-` statements are both used to assign values to variables.

 The diference is that the `<-` operator takes variables out of the context and stores them as a plain type. 

### Generalizing do-notation

Because the operators `>>` and `>>=` belong to the Monad type class, do-notation can be used for any type that has an instance of it.

Let's look at an example for the `Maybe` type that also had an instance of Monad.

In [ ]:
add1Maybe :: Num a => Maybe a -> Maybe a
add1Maybe maybeNum = do
    element <- maybeNum
    return $ element + 1

print $ add1Maybe $ Just 1
print $ add1Maybe Nothing

We see that even though we wrote `element + 1` this is handled correctly in case that `element` has the value Nothing.

You can use the same concept for the list data type that has also an instance of Monad type class. 

In [ ]:
add1List :: Num a => [a] -> [a]
add1List myList = do
    element <- myList
    return $ element + 1

print $ add1List [1..3]
print $ add1List []

When you run the code above you see that even though you perform the (+ 1) operation on a signle element you get a list returned. 

This is because you can view lists also as a context where the context means there are multiple possible values for a single tpye.

Now let's generalize these two functions by using the Monad type constraint in the type signature. 

In [ ]:
add1Monad :: (Num a, Monad m) => m a -> m a
add1Monad var = do
    raw <- var
    return $ raw + 1

print $ add1Monad [1..3]
print $ add1Monad $ Just 1

It works now for lists, maybe and other types that have an instance of the Monad type class.

## Monad examples

Another example of Monads is **list comprehension** which is an elegant way to create a list by specifying the element conditions inside the list. Here is a example:

In [ ]:
import Control.Monad ( guard )

list1 = [0 .. 9 :: Int]

add1Even :: [Int] -> [Int]
add1Even myList = do
  n <- myList
  let k = n + 1
  guard(even k)
  return k
  
add1Even' :: [Int] -> [Int]
add1Even' myList = [k | n <- myList, let k = n + 1, even k] 

print $ add1Even list1
print $ add1Even' list1

The `guard` function is basically a filter that passes only values of `k` which give `True` fot the expression inside the brackets. 

The function `add1Even` is just a desugared version of the function `add1Even'` and they work the same. 

In the next example lets define a function that returns a list of all prime numbers.

We can use now list comprehension together with recursion to define such a function.

In [ ]:
primeNumbers :: [Int]
primeNumbers = filterPrime [2..]
  where filterPrime (p:xs) =
          p : filterPrime [x | x <- xs, x `mod` p /= 0]

For our last example we create a parameterized type that we call `Writer`.
```haskell
data Writer a = Writer a String deriving Show
```
The parameter `a` can be for instance a numeric type as `Int` which we then use to do some computation with it. 

The idea of the Writer type is that while we do some computation to the numeric type we can log messages to the `String` parameter.

We will define the Monad instance for our Writer type in a way that will enable the concatenation of log messages.

<div class="alert alert-block alert-info">
    In later lessons we will learn about some new build-in Haskell types that also have a Monad instance. One of them is the build in `Writer` type that will be a generalized version of the `Writer` type that we create in this lesson.
</div>

When we deine the Monad instance we also need to declare instances for Functor and Applicative type classes. 

This needs to be done because Applicative is a superclass of Monad and Functor is a superclass of Applicative. 

In [ ]:
data Writer a = Writer a String
    deriving Show

bindWriter :: Writer a -> (a -> Writer b) -> Writer b
bindWriter (Writer a xs) f =
  let
    Writer b ys = f a
  in
    Writer b $ xs ++ ys

instance Functor Writer where  
    fmap f (Writer a xs) = Writer (f a) xs

instance Applicative Writer where
    pure a = Writer a []
    (Writer f xs) <*> (Writer a ys) =  
        let b = f a
        in Writer b (xs ++ ys)

instance Monad Writer where
    return a = Writer a ""
    (>>=) = bindWriter

We can now write a function that takes in three `Writer` variables, summs the numbers together and cocatenates the messages.

Also we create a `tell` function that just creates a Writer with the input message and an empty value.

In [ ]:
tell :: String -> Writer ()
tell msg = Writer () msg

tellAndSum :: Writer Int -> Writer Int -> Writer Int -> Writer Int
tellAndSum x y z = do
    tell "Sumed numbers: " 
    sumInts x y z

sumInts :: Monad m => m Int -> m Int -> m Int -> m Int
sumInts mx my mz = do
    k <- mx
    l <- my
    m <- mz
    let s = k + l + m
    return s

print $ tellAndSum (Writer 1 "1 ") (Writer 2 "2 ") (Writer 3 "3")

You might ask yourself how does this code work when it concatenates the messages? You have to look at the definition of the `>>` operator which is expressed with `>>=`. 

If you type `:i (>>)` you will see the comment `-- Defined in ‘GHC.Base’` which means it's defined in the source code of GHC [(1)](https://github.com/ghc/ghc/blob/master/libraries/base/GHC/Base.hs). 

As we said the minimal complete definition for Monad is only `>>=` so the `>>` operator is expressed by using `>>=`. 

Here is the `>>` definition from the source code:
```haskell
(>>) :: forall a b. m a -> m b -> m b
m >> k = m >>= \_ -> k
{-# INLINE (>>) #-}
```

In our Writer example we defined `>>=` (which represents `bindWriter`) in such a way that the message from `Writer a` gets concatenated with the message from `Writer b`. 

So when using the `>>` operator which actually uses the `>>=` operator, this concatenation of messages also happens:
```haskell
(Writer 1 "1 ") >> (Writer 2 "and 2") 
-- returns Writer 2 "1 and 2"
```

## More complex examples

As stated in the applicative lesson functions in general have also an instance of Monad type class. 

So you can re-write the fibonacci example using the bind operator.

In [ ]:
fibs = 0:1:(tail >>= zipWith (+)) fibs

print $ take 5 fibs

With the Monad operators it is possible also to express the operators for Functor and Applicative.

In [ ]:
fmapM :: Monad m => (a -> b) -> m a -> m b
fmapM func val = val >>= (\x -> return (func x))

appM :: Monad m => m (a -> b) -> m a -> m b
appM func val = func >>= (\f -> val >>= (\x -> return (f x)))

print $ fmapM (+1) $ Just 1
print $ appM (fmapM (+) $ Just 1) (Just 1)

This can be done also more elegantly with use of do-notation.

In [ ]:
fmapM :: (Monad m) => (a -> b) -> m a -> m b
fmapM f m1 = do 
    x1 <- m1
    return (f x1)

appM :: (Monad m) => m (a -> b) -> m a -> m b
appM m1 m2 = do 
    f <- m1
    x2 <- m2
    return (f x2) 

print $ fmapM (+1) $ Just 1
print $ appM (fmapM (+) $ Just 1) (Just 1)

Because of this, once you declare a monad instance for a type as we did for our `Writer` type, the functor and applicative instances are pre-defined.

In a later lesson about monad functions we well introduce the module **Control.Monad** that contains the functions `ap` and `liftM` which are defined as we defined `appM` and `fmapM`.

You still have to be careful when you define applicative and functor instances manually. Haskell does not check if they are the same as the actual definition with use of monad operators.

You could define for example the applicative instance such that it does not concatenate messages. Which is different as the definition we introduced in the previous chapter. 

```haskell
instance Applicative Writer where
    pure a = Writer a []
    (Writer f xs) <*> (Writer a ys) =  
        let b = f a
        in Writer b ys
```

This code would still compile but it is bad practice to define you applicative or functor instances such they work differently as the definitions with the use of monad operators.

## Monad laws

As for Functor and Applicative also the Monad type class has also its origins in mathematics and is defined with laws.

Monad has 3 laws. They are as follows:

- Identity (left):<br>`return a >>= f = f a`

- Identity (right):<br>`m >>= return = m`

- Associativity:<br>`(a >>= b) >>= c = a >>= (\x -> b x >>= c)`

The left identity law ensures that the bind operator `>>=` does nothing else as just take the input argument out of the context and apply the function to it.

The right identity law ensures that when using the return function it should not change anything except put the data back into context.

The associativity law ensures that the way we group the monadic actions doesn’t matter because `>>=` ensures that they’ll be evaluated in a consistent order.

Below is example code that proves these laws hold for the Maybe type which has an instance of Monad.

In [ ]:
-- Left identity law
((return 1 :: Maybe Int) >>= (\x -> return (x + 1) :: Maybe Int)) == (\x -> return (x + 1) :: Maybe Int) 1

-- Right identity law
((Just 1) >>= return) == Just 1

-- Associativity law
var1 :: Maybe Int
var1 = Just 1

add1, mult2 :: Int -> Maybe Int
add1 x = return (x + 1)
mult2 x = return (2 * x)

((var1 >>= add1) >>= mult2) == (var1 >>= (\x -> add1 x >>= mult2))

Let's look now at an example where we create a type and an monad instance for it that violates these laws.

In [ ]:
import Control.Monad (ap, liftM)

data NotOk a = NotOk a Bool deriving (Eq, Show)

bindNotOk :: NotOk a -> (a -> NotOk b) -> NotOk b
bindNotOk (NotOk a myBool) f =
  let
    NotOk b myBool' = f a
  in
    NotOk b $ not myBool

instance Functor NotOk where
    fmap = liftM

instance Applicative NotOk where
    pure = return
    (<*>) = ap

instance Monad NotOk where
  (>>=) = bindNotOk
  return a = NotOk a True

var1 :: NotOk Int
var1 = NotOk 1 True

add1, mult2 :: Int -> NotOk Int
add1 x = NotOk (x + 1) True
mult2 x = NotOk (2 * x) True

print $ ((return 1 :: NotOk Int) >>= add1) == add1 1
print $ (var1 >>= return) == var1
print $ ((var1 >>= add1) >>= mult2) == (var1 >>= (\x -> add1 x >>= mult2))

Same as for Functor and Applicative it is not obligatory to follow these laws in Haskell if you create a type and make an instance of Monad for it.

It is still good practice to follow them because:
- you can better reason about what your code is doing

- you can make use of other functions that work with the Monad type class

## Recap

In this lesson we've discussed:

- the motivation for introducing the Monad type 

- definition of the Monad type class

- the Monad operators and do-notation

- examples that show how to use the Monad type class

- laws that apply to the Monad type class